In [6]:
import pandas as pd
import numpy as np
from random import random

from CTL2.causal_tree_learn import CausalTree
from DGP.DGP import dgp2
from DGP.DGP import dgp4
from DGP.DGP import dgp4int
from DGP.DGP import dgp8

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

import matplotlib.pyplot as plt
import pickle as pkl
#from notify_run import Notify 
#notify = Notify()
import datetime

from utility.utility import transposer as tp
from utility.utility import writer
from utility.utility import reader

### Parameters:

### ATE2

In [4]:
def mc_ate4(n, tsize, var_e, reps, nomin_test):
    #true ATE
    ATE1 = np.ones(reps)*3
    ATE2 = np.ones(reps)*1
    ATE3 = np.ones(reps)*-2
    ATE4 = np.ones(reps)*-3
    #predicted ate
    ate1 = []
    ate2 = []
    ate3 = []
    ate4 = []
    #ITE dummy
    tauhat = np.zeros((tsize, 1))
    #true ITE

    for i in range(reps):
        x_train, x_test, y_train, y_test, treat_train, treat_test = dgp4int(n, tsize, var_e)
        tau = np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0)),3, 
                       np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0)),1, 
                                np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0)),-2,-3)))
        ctl = CausalTree(honest=True, weight=0.0, split_size=0.0, max_depth = 3) #which type of tree to call
        ctl.fit(x_train, y_train, treat_train, nomin_test = nomin_test) #select est size when fitting
        ctl.prune()
        ctl_predict = ctl.predict(x_test)
        #predicted ATE
        ate1 = np.append(ate1, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] >= 0))]))
        ate2 = np.append(ate2, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] < 0))]))
        ate3 = np.append(ate3, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] >= 0))]))
        ate4 = np.append(ate4, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] < 0))]))
        #predicted ITE
        tauhat = np.append(tauhat, ctl_predict.reshape(-1,1), axis = 1)

    mean_ate = np.array([np.mean(ate1), np.mean(ate2), np.mean(ate3), np.mean(ate4)])    
    mse_ate = np.array([mse(ATE1, ate1), mse(ATE2, ate2),mse(ATE3, ate3), mse(ATE4, ate4)])
    bias_ate = np.array([np.sum(np.subtract(ATE1,ate1))/reps, np.sum(np.subtract(ATE2,ate2))/reps, np.sum(np.subtract(ATE3,ate3))/reps, np.sum(np.subtract(ATE4,ate4))/reps])
    var_ate = np.array([np.var(ate1), np.var(ate2), np.var(ate3), np.var(ate4)])

    #TOTAL MSE's
    ind_var = np.var(tauhat[:, 1:], axis = 1)
    ind_mean = np.mean(tauhat, axis = 1)

    total_var = np.sum(ind_var)/tsize
    total_bias = np.sum(np.square(np.subtract(ind_mean, tau)))/tsize
    total_mse = total_var + total_bias

    results = np.concatenate([np.array([nomin_test]), mean_ate, bias_ate, var_ate, mse_ate, np.array([total_bias]), np.array([total_var]), np.array([total_mse])])
    return results

In [10]:
n = 1000
tsize = 500
reps = 500
test_sizes = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
res = []
var = [0.01, 1.0, 2.5]

print("start time: ", str(datetime.datetime.now())[11:16])

for v in var:
    df = pd.DataFrame()
    for size in test_sizes:
        results = mc_ate4(n, tsize, v, reps, size)
        df = pd.concat([df, pd.DataFrame(results).transpose()])
    df.columns=['test_sizes', 'Mean1', 'Mean2', 'Mean3', 'Mean4', 'MSE1', 'MSE2','MSE3', 'MSE4', 'BIAS1', 'BIAS2','BIAS3', 'BIAS4', 'VAR1', 'VAR2','VAR3', 'VAR4',  'MSE_T_BIAS', 'MSE_T_VAR', 'MSE_TOTAL']    
    res.append([v])
    res.append([df])
    
print("end time: ", str(datetime.datetime.now())[11:16])

start time:  23:26
end time:  23:27


In [11]:
res

[[0.01],
 [   test_sizes     Mean1     Mean2     Mean3     Mean4      MSE1      MSE2  \
  0         0.2  2.781741  1.534405 -2.136583 -2.181295  0.218259 -0.534405   
  0         0.3  2.514673  1.193286 -2.379273 -2.400994  0.485327 -0.193286   
  0         0.4  1.402720  1.543081 -2.150017 -2.151272  1.597280 -0.543081   
  0         0.5  1.922980  1.270820 -2.214503 -2.200395  1.077020 -0.270820   
  0         0.6  2.029387  1.546867 -2.474489 -2.579145  0.970613 -0.546867   
  0         0.7  2.091865  1.589949 -2.335762 -2.393147  0.908135 -0.589949   
  0         0.8  2.175513  1.582526 -2.425170 -2.438873  0.824487 -0.582526   
  
         MSE3      MSE4     BIAS1     BIAS2     BIAS3     BIAS4      VAR1  \
  0  0.136583 -0.818705  0.627526  0.612844  0.391660  0.408268  0.675164   
  0  0.379273 -0.599006  0.175736  0.382605  0.203657  0.182385  0.411278   
  0  0.150017 -0.848728  0.108328  0.086561  0.038510  0.038877  2.659630   
  0  0.214503 -0.799605  0.057167  0.114911  0.1